## Import packages

In [10]:
################### DO NOT ALTER ##################################################
from IPython.display import display, HTML
display(HTML("<style>.container { width:99% !important; }</style>"))

import os
import numpy as np
import pandas as pd
pd.options.mode.chained_assignment = None # avoids harmless warnings
import json
import matplotlib.pyplot as plt
import statsmodels.api as sm

from datetime import datetime

from backend_v2 import * # import all functions from backend

################### DO NOT ALTER ##################################################

## Import modeling database

In [11]:
# read the database
# STUDENT INPUT REQUIRED - Modify path directly below for location of the final_database.csv file on your laptop/desktop
db = pd.read_csv('/Users/minhphan/Downloads/Ekimetrics Project/Data Treatment/Processed Data/final_database.csv')

In [12]:
################### DO NOT ALTER ##################################################
# rename date column and turn to date format
db = db.rename(columns={'week_starting_date':'date'})
db['date'] = pd.to_datetime(db['date'])

## specify the name of the target and date columns of the database
target_column = 'sales'
date_column = 'date'

# view available variables
db.columns
################### DO NOT ALTER ##################################################

Index(['date', 'sales', 'bing_brand_search_clicks', 'bing_brand_search_spend',
       'blackfriday_dummy', 'covid_trend', 'google_brand_search_clicks',
       'google_brand_search_imp', 'google_brand_search_spend',
       'google_display_clicks', 'google_display_imp', 'google_display_spend',
       'google_nonbrand_search_clicks', 'google_nonbrand_search_imp',
       'google_nonbrand_search_spend', 'google_video_clicks',
       'google_video_imp', 'google_video_spend', 'gwp_units_distributed',
       'launch_fragrance_trend', 'launch_makeup_trend',
       'launch_skincare_trend', 'market_sales', 'mothersday_dummy',
       'nongoogle_display_spend', 'nongoogle_video_spend',
       'pinterest_awareness_imp', 'pinterest_awareness_spend',
       'pinterest_conversion_imp', 'pinterest_conversion_spend',
       'print_coop_spend', 'print_pure_spend', 'samples_quantity_distributed',
       'snap_awareness_imp', 'snap_awareness_spend', 'snap_conversion_imp',
       'snap_conversion_spend', 'ti

## Create custom variables

In [13]:
# at this step create: dummies, offsets, combined vars, etc.
################### DO NOT ALTER #######################################################################

# windows
db = window_feat(db, start_end_week_pairs = [['2020-11-02', '2020-12-21']], feat_name = 'covid_holiday_period',)
db = window_feat(db, start_end_week_pairs = [['2020-12-14', '2020-12-21']], feat_name = 'covid_xmas',)

# timesplit
db = timesplit_feat(db, feature='tv_coop_spend', split_week='2020-04-13', after=True)
db = timesplit_feat(db, feature='print_pure_spend', split_week='2020-04-13', after=True)
db = timesplit_feat(db, feature='print_pure_spend', split_week='2020-04-13', after=False)

# aggregated variables
db['total_product_launch'] = db['launch_fragrance_trend'] + db['launch_makeup_trend'] + db['launch_skincare_trend']
    
# dummy variables
db['week_before_laborday'] = (np.where((db['date']=='2020-08-24')|
                                (db['date']=='2019-08-26'),
                                1,
                                0
                                )
                        )

db['week_before_vday'] = (np.where((db['date']=='2020-02-03')|
                                (db['date']=='2019-02-04'),
                                1,
                                0
                                )
                        )

db['july_promo'] = (np.where((db['date']=='2020-07-06')|
                                (db['date']=='2019-07-08'),
                                1,
                                0
                                )
                        )

db['mothersday_2019'] = (np.where((db['date']=='2019-05-06'),
                                1,
                                0
                                )
                        )

db['blackfriday_2019'] = (np.where((db['date']=='2019-11-25'),
                                1,
                                0
                                )
                        )

db['friends_and_family_august_promo'] = (np.where((db['date']=='2019-08-05'),
                                1.5,
                                0
                                )
                        )

db['friends_and_family_august_promo'] = (np.where((db['date']=='2020-08-03'),
                                1,
                                db['friends_and_family_august_promo']
                                )
                        )

db['flash_sale_promo'] = (np.where((db['date']=='2019-03-18')|
                                   (db['date']=='2019-04-15')|
                                   (db['date']=='2020-06-08'),
                                1,
                                0
                                )
                        )
db.columns
#########################  DO NOT ALTER ABOVE #########################################################################################

Index(['date', 'sales', 'bing_brand_search_clicks', 'bing_brand_search_spend',
       'blackfriday_dummy', 'covid_trend', 'google_brand_search_clicks',
       'google_brand_search_imp', 'google_brand_search_spend',
       'google_display_clicks', 'google_display_imp', 'google_display_spend',
       'google_nonbrand_search_clicks', 'google_nonbrand_search_imp',
       'google_nonbrand_search_spend', 'google_video_clicks',
       'google_video_imp', 'google_video_spend', 'gwp_units_distributed',
       'launch_fragrance_trend', 'launch_makeup_trend',
       'launch_skincare_trend', 'market_sales', 'mothersday_dummy',
       'nongoogle_display_spend', 'nongoogle_video_spend',
       'pinterest_awareness_imp', 'pinterest_awareness_spend',
       'pinterest_conversion_imp', 'pinterest_conversion_spend',
       'print_coop_spend', 'print_pure_spend', 'samples_quantity_distributed',
       'snap_awareness_imp', 'snap_awareness_spend', 'snap_conversion_imp',
       'snap_conversion_spend', 'ti

In [34]:
db

,date,sales,bing_brand_search_clicks,bing_brand_search_spend,blackfriday_dummy,covid_trend,google_brand_search_clicks,google_brand_search_imp,google_brand_search_spend,google_display_clicks,...,total_product_launch,week_before_laborday,week_before_vday,july_promo,mothersday_2019,blackfriday_2019,friends_and_family_august_promo,flash_sale_promo,test,cyber_monday
0,2019-01-07,6.096763e+06,2900.876792,8535.213020,0,0.000000,6445.107608,2.255990e+05,5970.834004,192.450429,...,0.657775,0,0,0,0,0,0.0,0,0,0
1,2019-01-14,6.089210e+06,2256.867035,7213.173676,0,0.000000,5744.447122,2.077513e+05,5045.997395,77.508639,...,0.501940,0,0,0,0,0,0.0,0,0,0
2,2019-01-21,6.508507e+06,834.127984,7174.734540,0,0.000000,5925.447411,2.119892e+05,5019.107181,188.927309,...,0.601083,0,0,0,0,0,0.0,0,0,0
3,2019-01-28,7.022937e+06,1875.371520,7815.470738,0,0.000000,6411.197578,2.619131e+05,5467.336120,221.075779,...,0.023616,0,0,0,0,0,0.0,0,0,0
4,2019-02-04,1.963318e+07,2128.442802,9028.506866,0,0.000000,7195.532163,2.833293e+05,6315.919202,1185.529872,...,0.033966,0,1,0,0,0,0.0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99,2020-11-30,1.462212e+07,2824.703595,105341.195000,0,-19.458598,56896.185680,4.094003e+06,73691.750630,16974.832440,...,17.885437,0,0,0,0,0,0.0,0,1,1
100,2020-12-07,1.247291e+07,2513.715501,18308.944600,0,-19.254883,14948.157670,1.069767e+06,12808.077400,16779.739670,...,19.084376,0,0,0,0,0,0.0,0,0,0
101,2020-12-14,1.253015e+07,1940.213167,25012.460580,0,-19.926177,17575.964780,1.383143e+06,17497.542220,24742.431210,...,19.764961,0,0,0,0,0,0.0,0,0,0
102,2020-12-21,1.318809e+07,2012.609180,30650.488560,0,-36.497079,23008.175400,2.164692e+06,21441.641690,30633.087810,...,23.916584,0,0,0,0,0,0.0,0,0,0


In [25]:
# THIRD, make the custom seasonality variable here (use examples above for reference):
# STUDENT INPUT REQUIRED - Wait to create this seasonality variable until after analyzing your initial model results and...
#                          ...you have completed the optimization of the fbig_imp and outdoor_impressions adstock and lag settings





db['cyber_monday'] = (np.where((db['date'] == "2019-12-02") | (db["date"] == "2020-11-30"),
                                 1,
                                 0
                                 )
                        
                  )


## Build a Model and Evaluate

In [32]:
#########################  DO NOT ALTER   #########################################################################################
features_set = {
### Holidays ###
'blackfriday_2019':{},
'mothersday_2019':{'coeff':1.2e+07},
'week_before_vday':{'adstock':0.3,'coeff':9e+06}, #custom var
'july_promo':{},
'xmas_dummy':{},

### Market ###
'market_sales':{'coeff':2.2e-02},

### COVID  ###
'covid_holiday_period':{'coeff':-1.149319e+06},
'covid_xmas':{'coeff':-1.732296e+06},

### Product Launches ###
'total_product_launch':{'coeff':2840.642889},
    # 'launch_fragrance_trend':{},
    # 'launch_makeup_trend':{},
    # 'launch_skincare_trend':{},
    
### Promotions  ###
'gwp_units_distributed':{'coeff':9.734768},
'friends_and_family_august_promo':{'coeff':4e+06},
'flash_sale_promo':{},
    
### Traditional Media ###
'tv_pure_spend':{'lag':1, 'coeff':1.1},
'tv_coop_spend':{'coeff':4.5},
'tv_coop_spend_after_2020-04-13':{'coeff':-2.5},

'print_pure_spend_before_2020-04-13':{'coeff':1.3},
'print_pure_spend_after_2020-04-13':{'lag':1,'coeff':2},
'print_coop_spend':{'coeff':8.655333},
    
### Digital Media ###

# Search
'google_brand_search_imp':{'coeff':3.332862e-01},
'google_nonbrand_search_imp':{'coeff':0.665232},
'bing_brand_search_clicks':{'coeff':22},

# Social
'pinterest_awareness_imp':{'coeff':1.692867e-01},
'pinterest_conversion_imp':{'coeff':3.321005e-02},
'snap_awareness_imp':{'coeff':.04},
'snap_conversion_imp':{'coeff':.06},

# Video
'google_video_imp':{'coeff':0.0308512},
'youtube_video_imp':{'coeff':0.04582},
'nongoogle_video_spend':{'coeff':0.01132},

# Display
'google_display_imp':{'coeff':0.020604},
'nongoogle_display_spend':{'coeff':0.014703},
    
'covid_trend':{},
'samples_quantity_distributed':{},

#########################  DO NOT ALTER ABOVE #########################################################################################
    
    
##### USER INPUT SECTION #####
    

# FIRST test the lag and adstock (expected ranges: 0.2-0.6 for adstock, 0-2 for lag) for fbig_imp variable and make sure ...

# ...the outdoor_impressions variable further below remains commented out (inactive) while optimizing the fbig_imp variable.    

# STUDENT INPUT REQUIRED - test different adstock and lag settings to optimze the performance of the model and to maximize the value of the t stat

    'fbig_imp':{
    'adstock':0.6,
    'lag':0,
},
    
# SECOND test the lag and adstock (expected ranges: 0.2-0.6 for adstock, 0-2 for lag) for outdoor_impressions variable and make sure ...
# ...the fbig_imp variable has already been optimzed and remains uncommented (active) while optimizing the outdoor_impressions variable. 
# STUDENT INPUT REQUIRED - test different adstock and lag settings to optimze the performance of the model and to maximize the value of the t stat
    'outdoor_impressions':{
    'adstock':0.2,
    'lag':0,
},
    
## FOURTH, create the custom feature for the identified seasonality variable ##
    
# STUDENT INPUT REQUIRED - Wait to include this seasonality variable in the model feature set until after analyzing your initial model results and...
#                          ...you have completed the optimization of the fbig_imp and outdoor_impressions adstock and lag settings
#                          ...uncomment the line of code below and insert your actual variable name wehere it says variable name.
#                          ...make sure to keep the single quotes around your inserted variable name.    

    
'cyber_monday':{}
    
}

In [35]:
# run the model
ols_model, ols_lags, ols_adstocks, ols_model_kept_coeffs, ols_model_coeffs, best_database  = run_best_model(db, features_set, target_column)
model_df = model_output(ols_model_coeffs, ols_adstocks, ols_lags, ols_model, features_set)
model_df

,lag,adstock,saturation,coeff,t stat
blackfriday_2019,0,0.0,,4.069342e+06,4.235064
july_promo,0,0.0,,2.214241e+06,3.290393
xmas_dummy,0,0.0,,-1.040117e+06,-1.512357
flash_sale_promo,0,0.0,,2.517165e+06,4.607859
covid_trend,0,0.0,,5.759004e+04,8.064036
samples_quantity_distributed,0,0.0,,1.625912e+00,5.620277
fbig_imp,0,0.6,,3.713815e-02,3.590758
outdoor_impressions,0,0.2,,1.124465e-01,1.859349
cyber_monday,0,0.0,,3.240249e+06,4.657237
mothersday_2019,0,0.0,,1.200000e+07,0.000000


## To Paste in the Model sheet of the Analysis Tool

In [30]:
# paste the output to the Model sheet of the Analysis Tool
#STUDENT ACTION REQUIRED: Paste the results that are in your clipboard to the Model tab (position cursor in cell G5) in the provided Analysis Workbook spreadsheet
model_df.to_clipboard(header = False) # this line puts the output in the clipboard, you can then paste it where needed

## To Paste in the InputSheet sheet of the Analysis Tool

In [31]:
# Paste the output in the InputSheet sheet of the Analysis Tool.
#STUDENT ACTION REQUIRED: Paste the results that are in your clipboard to the InputSheet tab (position cursor in cell D3) in the provided Analysis Workbook spreadsheet
input_df = inputsheet(best_database, model_df)
input_df.to_clipboard(header = True, index = False) # this line puts the output in the clipboard, you can then paste it where needed